<a href="https://colab.research.google.com/github/gxilly/bsc_dpdm22/blob/main/%5Bdataprep1%5D_property_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import gspread as gs
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib

In [102]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [104]:
import os
os.chdir('/content/gdrive/MyDrive/bscdata')

!pwd #เช็ค paht ที่กำลังทำงานอยู่

/content/gdrive/MyDrive/bscdata


- ดูจากค่าอื่นๆ ที่ดินนี้อยู่ที่ไหน พื้นที่เท่าไหร่ ตร.ละเท่าไหร่ feature ต่างๆ เอาค่าพวกนี้มาสร้าง model ก่อน แล้วเอาค่าที่มีอยู่ มาทำนายค่าที่หายไป ## ดีเยี่ยม!!
- 0 เกิน 0 ขาด ส่วนใหญ่น่าจะตกแค่ตัวเดียว ต่างกัน 10 เท่า
ควรเอาราคาสองอันมาหารกัน (ประเมิน/ตั้งขาย, ตั้งขาย/ประเมิน) ห่างกัน 9-10,0.1 

# import ข้อมูล

## House

In [5]:
# open file
house = pd.read_excel('คำตอบนักประเมินaj.xlsx',sheet_name=0)
print(house.shape) # shown number of (row, column)
house.head()

(190, 14)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,100.0
1,2,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/495,บ้านเดี่ยว,มั่นใจ,5900000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN


In [116]:
link = 'https://docs.google.com/spreadsheets/d/1hRZ_CnkB0tZDDgW_pXS-aw61X1PIxZDdeT1q2SXXIIs/export?format=xlsx'
L = pd.read_excel(link,sheet_name='land')
L

ValueError: ignored

In [109]:
gc = gs.service_account(filename='คำตอบนักประเมิน_update')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1hRZ_CnkB0tZDDgW_pXS-aw61X1PIxZDdeT1q2SXXIIs/edit#gid=905447057')

ws = sh.worksheet('คำตอบนักประเมิน_update')
df = pd.DataFrame(ws.get_all_records())
df.head()

AttributeError: ignored

## Land (ที่ดิน)

In [6]:
land = pd.read_excel('คำตอบนักประเมินaj.xlsx',sheet_name=1)

(210, 13)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,1800000.0,"8,000-25,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,100.0
1,2,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,2000000.0,"ราคาเสนอขาย 7,000-12,0000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN
2,4,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,3000000.0,12000-18000,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,NaN
3,1,https://nayoo.co/khonkaen/posts/1711,ที่ดิน,มั่นใจ,14900000,8000000.0,"25,000-80,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN
4,2,https://nayoo.co/khonkaen/posts/1711,ที่ดิน,มั่นใจ,14900000,12000000.0,"ราคาเสนอขาย 100,000-150,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN


## Condo

In [7]:
condo = pd.read_excel('คำตอบนักประเมินaj.xlsx',sheet_name=2)

(10, 11)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10
0,1,32.0,คอนโด,มั่นใจ,3500000.0,3500000.0,"60,000-130,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1,1,36.0,คอนโด,มั่นใจ,1930000.0,2200000.0,"50,000-90,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
2,1,603.0,คอนโด,มั่นใจ,2438000.0,2800000.0,"35,000-80,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,1,1395.0,คอนโด,มั่นใจ,1600000.0,2300000.0,"30,000-80,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,1443.0,คอนโด,มั่นใจ,1860000.0,1800000.0,"30,000-80,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี


## Town home

In [8]:
tw = pd.read_excel('คำตอบนักประเมินaj.xlsx',sheet_name=3)

(240, 13)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,https://nayoo.co/khonkaen/posts/548,ทาวน์โฮม,มั่นใจ,1045000.0,700000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,120.0
1,2,https://nayoo.co/khonkaen/posts/548,ทาวน์โฮม,มั่นใจ,1045000.0,1500000.0,ราคาซื้อขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/549,ทาวน์โฮม,มั่นใจ,935000.0,700000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/549,ทาวน์โฮม,มั่นใจ,935000.0,1500000.0,ราคาเสนอขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/579,ทาวน์โฮม,มั่นใจ,3520000.0,3500000.0,"8,000-80,000",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN


## Shop house (อาคารพาณิชย์)

In [9]:
sh= pd.read_excel('คำตอบนักประเมินaj.xlsx',sheet_name=4)

(20, 14)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,https://nayoo.co/khonkaen/posts/6,อาคารพาณิชย์/สำนักงาน,มั่นใจ,7000000.0,6000000.0,"8,000-30,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,NaN,10.0
1,2,https://nayoo.co/khonkaen/posts/6,อาคารพาณิชย์/สำนักงาน,มั่นใจ,7000000.0,8000000.0,"ราคาเสนอขาย 4,000,000-6,000,000 บาท/คูหา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/17,อาคารพาณิชย์/สำนักงาน,มั่นใจ,12000000.0,12000000.0,"8,000-40,000",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/17,อาคารพาณิชย์/สำนักงาน,มั่นใจ,12000000.0,12000000.0,ราคาเสนอขาย 4 - 5 ล้านบาท/คูหา,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/21,อาคารพาณิชย์/สำนักงาน,มั่นใจ,22500000.0,13500000.0,"8,000-50,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,NaN


# สรุปข้อมูลของนักประเมินแต่ละคน โดยพิจารณาจาก ความมั่นใจในการประเมิน ประเมินราคาซื้อ-ขาย ให้คะแนนทรัพย์

## House

In [10]:
house['ID ผู้ประเมิน'].value_counts()

1    100
2     90
Name: ID ผู้ประเมิน, dtype: int64

In [11]:
est1 = house[house['ID ผู้ประเมิน']==1]

In [12]:
data01 = est1[['ID ผู้ประเมิน','ความมั่นใจในการเมินราคาทรัพย์','ประเมินราคาซื้อ-ขาย','ให้คะแนนทรัพย์']]
data01.head(5)

,ID ผู้ประเมิน,ความมั่นใจในการเมินราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ให้คะแนนทรัพย์
0,1,มั่นใจ,2500000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,1,มั่นใจ,2200000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,มั่นใจ,5000000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
6,1,มั่นใจ,3500000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
8,1,มั่นใจ,2500000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [13]:
est1.describe()

,ID ผู้ประเมิน,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
count,100.0,1.000000e+02,1.000000e+02,0.0,0.0,0.0,0.0,1.0
mean,1.0,4.316360e+06,3.938500e+06,NaN,NaN,NaN,NaN,100.0
std,0.0,2.865391e+06,2.368553e+06,NaN,NaN,NaN,NaN,NaN
min,1.0,3.900000e+05,6.000000e+05,NaN,NaN,NaN,NaN,100.0
25%,1.0,2.892750e+06,2.775000e+06,NaN,NaN,NaN,NaN,100.0
50%,1.0,3.962000e+06,3.600000e+06,NaN,NaN,NaN,NaN,100.0
75%,1.0,5.000000e+06,4.500000e+06,NaN,NaN,NaN,NaN,100.0
max,1.0,2.094900e+07,1.600000e+07,NaN,NaN,NaN,NaN,100.0


In [14]:
est1['ให้คะแนนทรัพย์'].value_counts()

2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี    54
3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน     43
4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน       2
1 ขายไม่ได้แน่นอน                      1
Name: ให้คะแนนทรัพย์, dtype: int64

In [15]:
est1['ความมั่นใจในการเมินราคาทรัพย์'].value_counts()

มั่นใจ    100
Name: ความมั่นใจในการเมินราคาทรัพย์, dtype: int64

In [16]:
est2 = house[house['ID ผู้ประเมิน']==2]

In [17]:
data02 =est2[['ID ผู้ประเมิน','ความมั่นใจในการเมินราคาทรัพย์','ประเมินราคาซื้อ-ขาย','ให้คะแนนทรัพย์']]
data02.head(5)

,ID ผู้ประเมิน,ความมั่นใจในการเมินราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ให้คะแนนทรัพย์
1,2,มั่นใจ,2700000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
3,2,มั่นใจ,2400000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
5,2,มั่นใจ,9000000.0,5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน
7,2,มั่นใจ,3700000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
9,2,มั่นใจ,2500000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [18]:
est2.describe()

,ID ผู้ประเมิน,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
count,90.0,9.000000e+01,8.900000e+01,0.0,0.0,0.0,0.0,0.0
mean,2.0,4.240733e+06,4.153258e+06,NaN,NaN,NaN,NaN,NaN
std,0.0,2.714815e+06,2.386901e+06,NaN,NaN,NaN,NaN,NaN
min,2.0,3.900000e+05,9.900000e+05,NaN,NaN,NaN,NaN,NaN
25%,2.0,2.860000e+06,2.600000e+06,NaN,NaN,NaN,NaN,NaN
50%,2.0,4.000000e+06,3.900000e+06,NaN,NaN,NaN,NaN,NaN
75%,2.0,5.000000e+06,4.900000e+06,NaN,NaN,NaN,NaN,NaN
max,2.0,2.094900e+07,1.700000e+07,NaN,NaN,NaN,NaN,NaN


In [19]:
est2['ให้คะแนนทรัพย์'].value_counts()

3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน     64
4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน      23
5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน      1
2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี     1
Name: ให้คะแนนทรัพย์, dtype: int64

In [20]:
est2['ความมั่นใจในการเมินราคาทรัพย์'].value_counts()

มั่นใจ       89
ไม่มั่นใจ     1
Name: ความมั่นใจในการเมินราคาทรัพย์, dtype: int64

## Land

In [21]:
land['ID ผู้ประเมิน'].value_counts() 

1    100
2    100
4     10
Name: ID ผู้ประเมิน, dtype: int64

ใช้แค่ ID 01 กับ 02

In [22]:
est_1l = land[land['ID ผู้ประเมิน']==1]

In [23]:
est_2l = land[land['ID ผู้ประเมิน']==2]

## Condo

In [24]:
condo['ID ผู้ประเมิน'].value_counts() 

1    10
Name: ID ผู้ประเมิน, dtype: int64

ใน condo มีผู้ประเมินคนเดียว คือ ID 01

## Town home

In [25]:
tw['ID ผู้ประเมิน'].value_counts() 

1    120
2    120
Name: ID ผู้ประเมิน, dtype: int64

In [26]:
est_1t = tw[tw['ID ผู้ประเมิน']==1]

In [27]:
est_2t =  tw[tw['ID ผู้ประเมิน']==2]

## Shop house

In [28]:
sh['ID ผู้ประเมิน'].value_counts() 

1    10
2    10
Name: ID ผู้ประเมิน, dtype: int64

In [29]:
est_1s = sh[sh['ID ผู้ประเมิน']==1]

In [30]:
est_2s =  sh[sh['ID ผู้ประเมิน']==2]

# HW1

#เปรียบเทียบความแตกต่างในการประเมิณราคาทรัพย์แต่ละทรัพย์ของนักประเมินแต่ละคนและของราคาตั้งขาย

In [31]:
from sklearn.metrics import mean_absolute_percentage_error as mape #mape

## House

In [32]:
est1 = est1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])

In [33]:
est1

,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,100.0
2,1,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/495,บ้านเดี่ยว,มั่นใจ,5900000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN
6,1,https://nayoo.co/khonkaen/posts/564,บ้านเดี่ยว,มั่นใจ,4400000.0,3500000.0,"5,000-18,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
8,1,https://nayoo.co/khonkaen/posts/566,บ้านเดี่ยว,มั่นใจ,2750000.0,2500000.0,"5,000-25,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1,https://nayoo.co/khonkaen/posts/19495,บ้านเดี่ยว,มั่นใจ,5500000.0,4500000.0,"3,000-30,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
156,1,https://nayoo.co/khonkaen/posts/19496,บ้านเดี่ยว,มั่นใจ,7911000.0,5200000.0,"3,000-30,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
157,1,https://nayoo.co/khonkaen/posts/20,บ้านเดี่ยว,มั่นใจ,2000000.0,3600000.0,"3,000-30,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
158,1,https://nayoo.co/khonkaen/posts/31,บ้านเดี่ยว,มั่นใจ,4000000.0,4000000.0,"3,000-35,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN


In [34]:
est1MAPE = mape(est1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(est1['ราคาทรัพย์']))

In [35]:
est1MAPE 

0.1299616960144211

In [36]:
est2['ประเมินโดย 1'] = est2['post id'].map(est1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est2.head(3)

<ipython-input-36-3501914803de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est2['ประเมินโดย 1'] = est2['post id'].map(est1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN,NaN,2500000.0
3,2,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,2200000.0
5,2,https://nayoo.co/khonkaen/posts/495,บ้านเดี่ยว,มั่นใจ,5900000.0,9000000.0,"โซนนี้ราคาซื้อขาย ประมาณ 10,000-20,000 บาท/ตาร...",NaN,5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน,NaN,NaN,NaN,NaN,NaN,5000000.0


In [37]:
est2.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [38]:
est2

,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินโดย 2,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN,NaN,2500000.0
3,2,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,2200000.0
5,2,https://nayoo.co/khonkaen/posts/495,บ้านเดี่ยว,มั่นใจ,5900000.0,9000000.0,"โซนนี้ราคาซื้อขาย ประมาณ 10,000-20,000 บาท/ตาร...",NaN,5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน,NaN,NaN,NaN,NaN,NaN,5000000.0
7,2,https://nayoo.co/khonkaen/posts/564,บ้านเดี่ยว,มั่นใจ,4400000.0,3700000.0,"เสนอขาย 3,500,000 - 4,200,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,3500000.0
9,2,https://nayoo.co/khonkaen/posts/566,บ้านเดี่ยว,มั่นใจ,2750000.0,2500000.0,"ราคาเสนอขาย 2,000,0000 - 2,800,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,2500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,2,https://nayoo.co/khonkaen/posts/19495,บ้านเดี่ยว,มั่นใจ,5500000.0,5000000.0,"ราคาเสนอขาย 20,000-30,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,4500000.0
186,2,https://nayoo.co/khonkaen/posts/19496,บ้านเดี่ยว,มั่นใจ,7911000.0,7400000.0,"ราคาเสนอขาย 20,000 - 30,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,5200000.0
187,2,https://nayoo.co/khonkaen/posts/20,บ้านเดี่ยว,มั่นใจ,2000000.0,2500000.0,ราคาเสนอขาย 2.2-3.0 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,3600000.0
188,2,https://nayoo.co/khonkaen/posts/31,บ้านเดี่ยว,มั่นใจ,4000000.0,3900000.0,ราคาเสนอขาย 3.5-4.0ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,4000000.0


In [39]:
ass_h = est2[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_h = ass_h.dropna(subset=['ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2'])
ass_h

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/25,2700000.0,2500000.0,2700000.0
3,https://nayoo.co/khonkaen/posts/33,2600000.0,2200000.0,2400000.0
5,https://nayoo.co/khonkaen/posts/495,5900000.0,5000000.0,9000000.0
7,https://nayoo.co/khonkaen/posts/564,4400000.0,3500000.0,3700000.0
9,https://nayoo.co/khonkaen/posts/566,2750000.0,2500000.0,2500000.0
...,...,...,...,...
185,https://nayoo.co/khonkaen/posts/19495,5500000.0,4500000.0,5000000.0
186,https://nayoo.co/khonkaen/posts/19496,7911000.0,5200000.0,7400000.0
187,https://nayoo.co/khonkaen/posts/20,2000000.0,3600000.0,2500000.0
188,https://nayoo.co/khonkaen/posts/31,4000000.0,4000000.0,3900000.0


เช็คและแก้ missing

In [40]:
ass_h.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

In [41]:
# ass_h = ass_h.fillna(value={'ประเมินโดย 2':est2['ประเมินโดย 2'].mean()}) #ควร drop

In [42]:
#ass_h = ass_h.dropna(subset=['ราคาทรัพย์','ประเมินโดย 2'])

ดูจากค่าอื่นๆ ที่ดินนี้อยู่ที่ไหน พื้นที่เท่าไหร่ ตร.ละเท่าไหร่ feature ต่างๆ เอาค่าพวกนี้มาสร้าง model ก่อน แล้วเอาค่าที่มีอยู่ มาทำนายค่าที่หายไป ## ดีเยี่ยม!!

In [43]:
#ass_h.isnull().any()

### MAPE

In [44]:
actual_h = pd.to_numeric(ass_h['ราคาทรัพย์'])
pred_1h = ass_h['ประเมินโดย 1']
pred_2h = ass_h['ประเมินโดย 2']

In [45]:
mph1 = mape(actual_h,pred_1h)
mph2 = mape(actual_h,pred_2h)
mph12 = mape(pred_1h,pred_2h)
mph21 = mape(pred_2h,pred_1h)

In [46]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mph1*100:.2f}%\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mph2*100:.2f}%\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mph12*100:.2f}%\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ {mph21*100:.2f}%')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 12.32%
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 10.42%
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 13.63%
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ 11.58%


In [47]:
temp = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=0)
id1 = temp[temp['ID ผู้ประเมิน']==1]
id1 = id1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
id2 = temp[temp['ID ผู้ประเมิน']==2]
id2 = id2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
id1MAPE = mape(id1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(id1['ราคาทรัพย์']))
id2MAPE = mape(id2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(id2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {id1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {id2MAPE*100:.2f}%')
id2['ประเมิน2'] = id2['post id'].map(id1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_d = id2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2two = mape(id2_d['ประเมินราคาซื้อ-ขาย'],id2_d['ประเมิน2'])
two2one = mape(id2_d['ประเมิน2'],id2_d['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2two)+(two2one))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_d)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 13.00%
นักประเมิน คนที่ 2 = 8.52%
ความผิดพลาดระหว่างนักประเมินทั้ง 12.60%
จำนวนทรัพย์ทั้งหมด 89 ทรัพย์


## Land

In [48]:
est_2l['ประเมินโดย 1'] = est_2l['post id'].map(est_1l.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est_2l.head(3)

<ipython-input-48-8ea92906a684>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_2l['ประเมินโดย 1'] = est_2l['post id'].map(est_1l.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,2000000.0,"ราคาเสนอขาย 7,000-12,0000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,1800000.0
4,2,https://nayoo.co/khonkaen/posts/1711,ที่ดิน,มั่นใจ,14900000,12000000.0,"ราคาเสนอขาย 100,000-150,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,8000000.0
7,2,https://nayoo.co/khonkaen/posts/1726,ที่ดิน,มั่นใจ,90000000,70000000.0,"30,000-40,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,18000000.0


In [49]:
est_2l.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [50]:
ass_l = est_2l[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_l

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/1033,2800000,1800000.0,2000000.0
4,https://nayoo.co/khonkaen/posts/1711,14900000,8000000.0,12000000.0
7,https://nayoo.co/khonkaen/posts/1726,90000000,18000000.0,70000000.0
10,https://nayoo.co/khonkaen/posts/1732,65000000,35000000.0,60000000.0
13,https://nayoo.co/khonkaen/posts/1866,3000000,2200000.0,2500000.0
...,...,...,...,...
200,https://nayoo.co/khonkaen/posts/14539,5000000,4000000.0,4000000.0
202,https://nayoo.co/khonkaen/posts/16072,750000,500000.0,750000.0
204,https://nayoo.co/khonkaen/posts/17093,2650000,2000000.0,2500000.0
206,https://nayoo.co/khonkaen/posts/17112,1300000,1500000.0,1500000.0


เช็คและแก้ missing

In [51]:
ass_l.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [52]:
actual_l = ass_l[['ราคาทรัพย์']]
pred_1l = ass_l[['ประเมินโดย 1']]
pred_2l = ass_l[['ประเมินโดย 2']]

In [53]:
mpl1 = mape(actual_l,pred_1l)
mpl2 = mape(actual_l,pred_2l)
mpl12 = mape(pred_1l,pred_2l)
mpl21 = mape(pred_2l,pred_1l)

In [54]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpl1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mpl2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mpl12:.2f}\n MAPEราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ {mpl21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.41
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.40
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.74
 MAPEราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ 0.43


In [55]:
temp = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=1)
idl1 = temp[temp['ID ผู้ประเมิน']==1]
idl1 = idl1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idl2 = temp[temp['ID ผู้ประเมิน']==2]
idl2 = idl2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idl1MAPE = mape(idl1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idl1['ราคาทรัพย์']))
idl2MAPE = mape(idl2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idl2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idl1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {idl2MAPE*100:.2f}%')
idl2['ประเมิน2'] = idl2['post id'].map(idl1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_l = idl2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twol = mape(id2_l['ประเมินราคาซื้อ-ขาย'],id2_l['ประเมิน2'])
two2onel = mape(id2_l['ประเมิน2'],id2_l['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twol)+(two2onel))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_l)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 83.48%
นักประเมิน คนที่ 2 = 48.38%
ความผิดพลาดระหว่างนักประเมินทั้ง 58.41%
จำนวนทรัพย์ทั้งหมด 100 ทรัพย์


## Condo

In [56]:
condo.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 1'}, inplace=True)

In [57]:
ass_c = condo[['post id','ราคาทรัพย์','ประเมินโดย 1']]
ass_c

,post id,ราคาทรัพย์,ประเมินโดย 1
0,32.0,3500000.0,3500000.0
1,36.0,1930000.0,2200000.0
2,603.0,2438000.0,2800000.0
3,1395.0,1600000.0,2300000.0
4,1443.0,1860000.0,1800000.0
5,1609.0,1490000.0,1500000.0
6,1640.0,1280000.0,1600000.0
7,1641.0,4290000.0,2700000.0
8,1650.0,1700000.0,1550000.0
9,1840.0,3790000.0,2800000.0


เช็คและแก้ missing

In [58]:
ass_c.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
dtype: bool

### MAPE

In [59]:
actual_c = ass_c[['ราคาทรัพย์']]
pred_1c = ass_c[['ประเมินโดย 1']]

In [60]:
mpc1 = mape(actual_c,pred_1c)

In [61]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpc1:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.17


In [62]:
temp = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=2)
idc1 = temp[temp['ID ผู้ประเมิน']==1]
idc1 = idc1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])

idc1MAPE = mape(idc1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idc1['ราคาทรัพย์']))

print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idc1MAPE*100:.2f}%')


print(f'จำนวนทรัพย์ทั้งหมด {len(idc1)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 18.36%
จำนวนทรัพย์ทั้งหมด 10 ทรัพย์


## Town home

In [63]:
est_2t['ประเมินโดย 1'] = est_2t['post id'].map(est_1t.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est_2t.head(3)

<ipython-input-63-309d826ee320>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_2t['ประเมินโดย 1'] = est_2t['post id'].map(est_1t.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/548,ทาวน์โฮม,มั่นใจ,1045000.0,1500000.0,ราคาซื้อขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN,700000.0
3,2,https://nayoo.co/khonkaen/posts/549,ทาวน์โฮม,มั่นใจ,935000.0,1500000.0,ราคาเสนอขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN,700000.0
5,2,https://nayoo.co/khonkaen/posts/579,ทาวน์โฮม,มั่นใจ,3520000.0,4000000.0,ราคาเสนอขาย 3-5 ล้านบาท/คูหา,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,3500000.0


In [64]:
est_2t.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [65]:
ass_th = est_2t[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_th

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/548,1045000.0,700000.0,1500000.0
3,https://nayoo.co/khonkaen/posts/549,935000.0,700000.0,1500000.0
5,https://nayoo.co/khonkaen/posts/579,3520000.0,3500000.0,4000000.0
7,https://nayoo.co/khonkaen/posts/580,3850000.0,3500000.0,3200000.0
9,https://nayoo.co/khonkaen/posts/582,990000.0,1000000.0,800000.0
...,...,...,...,...
235,https://nayoo.co/khonkaen/posts/18547,1430000.0,1000000.0,1200000.0
236,https://nayoo.co/khonkaen/posts/18551,1980000.0,1500000.0,1700000.0
237,https://nayoo.co/khonkaen/posts/19434,2000000.0,1600000.0,1900000.0
238,https://nayoo.co/khonkaen/posts/19501,1790000.0,1600000.0,1700000.0


เช็คและแก้ missing

In [66]:
ass_th.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [67]:
actual_th = ass_th[['ราคาทรัพย์']]
pred_1th = ass_th[['ประเมินโดย 1']]
pred_2th = ass_th[['ประเมินโดย 2']]

In [68]:
mpth1 = mape(actual_th,pred_1th)
mpth2 = mape(actual_th,pred_2th)
mpth12 = mape(pred_1th,pred_2th)
mpth21 = mape(pred_2th,pred_1th)

In [69]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpth1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mpth2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mpth12:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ {mpth21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.11
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.09
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.11
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ 0.11


In [70]:
temp = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=3)
idt1 = temp[temp['ID ผู้ประเมิน']==1]
idt1 = idt1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idt2 = temp[temp['ID ผู้ประเมิน']==2]
idt2 = idt2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idt1MAPE = mape(idt1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idt1['ราคาทรัพย์']))
idt2MAPE = mape(idt2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idt2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idt1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {idt2MAPE*100:.2f}%')
idt2['ประเมิน2'] = idt2['post id'].map(idt1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_t = idt2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twot = mape(id2_t['ประเมินราคาซื้อ-ขาย'],id2_t['ประเมิน2'])
two2onet = mape(id2_t['ประเมิน2'],id2_t['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twot)+(two2onet))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_t)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 16.21%
นักประเมิน คนที่ 2 = 12.75%
ความผิดพลาดระหว่างนักประเมินทั้ง 10.92%
จำนวนทรัพย์ทั้งหมด 120 ทรัพย์


## Shop house

In [71]:
est_2s['ประเมินโดย 1'] = est_2s['post id'].map(est_1s.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est_2s.head(3)

<ipython-input-71-f42338c5bcdb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_2s['ประเมินโดย 1'] = est_2s['post id'].map(est_1s.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/6,อาคารพาณิชย์/สำนักงาน,มั่นใจ,7000000.0,8000000.0,"ราคาเสนอขาย 4,000,000-6,000,000 บาท/คูหา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,6000000.0
3,2,https://nayoo.co/khonkaen/posts/17,อาคารพาณิชย์/สำนักงาน,มั่นใจ,12000000.0,12000000.0,ราคาเสนอขาย 4 - 5 ล้านบาท/คูหา,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,NaN,12000000.0
5,2,https://nayoo.co/khonkaen/posts/21,อาคารพาณิชย์/สำนักงาน,มั่นใจ,22500000.0,20000000.0,ราคาเสนอขาย 6-8 ล้านบาท / คูหา,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,NaN,NaN,13500000.0


In [72]:
est_2s.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [73]:
ass_sh = est_2s[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_sh

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/6,7000000.0,6000000.0,8000000.0
3,https://nayoo.co/khonkaen/posts/17,12000000.0,12000000.0,12000000.0
5,https://nayoo.co/khonkaen/posts/21,22500000.0,13500000.0,20000000.0
7,https://nayoo.co/khonkaen/posts/22,3500000.0,4000000.0,4000000.0
9,https://nayoo.co/khonkaen/posts/24,3900000.0,2200000.0,3000000.0
11,https://nayoo.co/khonkaen/posts/30,4800000.0,3600000.0,4000000.0
13,https://nayoo.co/khonkaen/posts/35,2400000.0,2500000.0,2100000.0
15,https://nayoo.co/khonkaen/posts/497,5000000.0,3800000.0,5000000.0
17,https://nayoo.co/khonkaen/posts/498,7000000.0,7000000.0,8000000.0
19,https://nayoo.co/khonkaen/posts/499,9000000.0,8000000.0,10000000.0


เช็คและแก้ missing

In [74]:
ass_sh.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [75]:
actual_sh = ass_sh[['ราคาทรัพย์']]
pred_1sh = ass_sh[['ประเมินโดย 1']]
pred_2sh = ass_sh[['ประเมินโดย 2']]

In [76]:
mpsh1 = mape(actual_sh,pred_1sh)
mpsh2 = mape(actual_sh,pred_2sh)
mpsh12 = mape(pred_1sh,pred_2sh)
mpsh21 = mape(pred_2sh,pred_1sh)

In [77]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpsh1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mpsh2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mpsh12:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 2 คือ {mpsh21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.18
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.12
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.22
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 2 คือ 0.17


In [78]:
temp = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=4)
ids1 = temp[temp['ID ผู้ประเมิน']==1]
ids1 = ids1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
ids2 = temp[temp['ID ผู้ประเมิน']==2]
ids2 = ids2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
ids1MAPE = mape(ids1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(ids1['ราคาทรัพย์']))
ids2MAPE = mape(ids2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(ids2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {ids1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {ids2MAPE*100:.2f}%')
ids2['ประเมิน2'] = ids2['post id'].map(ids1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_s = idt2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twos = mape(id2_s['ประเมินราคาซื้อ-ขาย'],id2_s['ประเมิน2'])
two2ones = mape(id2_s['ประเมิน2'],id2_s['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twos)+(two2ones))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_s)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 25.45%
นักประเมิน คนที่ 2 = 12.43%
ความผิดพลาดระหว่างนักประเมินทั้ง 10.92%
จำนวนทรัพย์ทั้งหมด 120 ทรัพย์


# HW2

# เขียน function ตรวจสอบค่าที่เติม 0 ผิด

Check1 = list()
for row in ass_h.iterrows() :
    print(row)

In [79]:
max(10,2)

10

In [80]:
def check_v(in1,in2):
    return max(in1/in2,in2/in1)

In [81]:
check_v(100,12)

8.333333333333334

In [82]:
check_v(100,1200)

12.0

# สร้าง column ใหม่ ลงในตาราง home โดย ค่าใน column คือค่า จาก function check(ราคาตั้งขาย,ราคาประเมิน1)

## House

In [83]:
ass_h['Check1'] = ass_h.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)
ass_h['Check2'] = ass_h.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)

In [84]:
ass_h.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
187,https://nayoo.co/khonkaen/posts/20,2000000.0,3600000.0,2500000.0,1.800000,1.250000
75,https://nayoo.co/khonkaen/posts/1,390000.0,600000.0,1300000.0,1.538462,3.333333
177,https://nayoo.co/khonkaen/posts/19450,2100000.0,3200000.0,2200000.0,1.523810,1.047619
186,https://nayoo.co/khonkaen/posts/19496,7911000.0,5200000.0,7400000.0,1.521346,1.069054
101,https://nayoo.co/khonkaen/posts/6010,2860000.0,4200000.0,3000000.0,1.468531,1.048951
...,...,...,...,...,...,...
167,https://nayoo.co/khonkaen/posts/9141,3500000.0,3500000.0,3600000.0,1.000000,1.028571
51,https://nayoo.co/khonkaen/posts/1149,4000000.0,4000000.0,4500000.0,1.000000,1.125000
63,https://nayoo.co/khonkaen/posts/2033,1200000.0,1200000.0,1000000.0,1.000000,1.200000
188,https://nayoo.co/khonkaen/posts/31,4000000.0,4000000.0,3900000.0,1.000000,1.025641


In [85]:
ass_h.sort_values(by=['Check2'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
75,https://nayoo.co/khonkaen/posts/1,390000.0,600000.0,1300000.0,1.538462,3.333333
5,https://nayoo.co/khonkaen/posts/495,5900000.0,5000000.0,9000000.0,1.180000,1.525424
111,https://nayoo.co/khonkaen/posts/6045,2915000.0,2000000.0,2000000.0,1.457500,1.457500
83,https://nayoo.co/khonkaen/posts/3721,3100000.0,3500000.0,4000000.0,1.129032,1.290323
187,https://nayoo.co/khonkaen/posts/20,2000000.0,3600000.0,2500000.0,1.800000,1.250000
...,...,...,...,...,...,...
169,https://nayoo.co/khonkaen/posts/11264,6500000.0,5600000.0,6500000.0,1.160714,1.000000
69,https://nayoo.co/khonkaen/posts/2262,3200000.0,3300000.0,3200000.0,1.031250,1.000000
57,https://nayoo.co/khonkaen/posts/1895,1200000.0,1500000.0,1200000.0,1.250000,1.000000
37,https://nayoo.co/khonkaen/posts/760,1800000.0,1700000.0,1800000.0,1.058824,1.000000


### MAPE ใหม่

In [86]:
temp = pd.read_excel('คำตอบนักประเมินแก้ไข.xlsx',sheet_name=0)
id1 = temp[temp['ID ผู้ประเมิน']==1]
id1 = id1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
id2 = temp[temp['ID ผู้ประเมิน']==2]
id2 = id2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
id1MAPE = mape(id1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(id1['ราคาทรัพย์']))
id2MAPE = mape(id2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(id2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {id1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {id2MAPE*100:.2f}%')
id2['ประเมิน2'] = id2['post id'].map(id1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_d = id2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2two = mape(id2_d['ประเมินราคาซื้อ-ขาย'],id2_d['ประเมิน2'])
two2one = mape(id2_d['ประเมิน2'],id2_d['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2two)+(two2one))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_d)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 13.75%
นักประเมิน คนที่ 2 = 9.51%
ความผิดพลาดระหว่างนักประเมินทั้ง 13.54%
จำนวนทรัพย์ทั้งหมด 61 ทรัพย์


## Land

In [87]:
ass_l['Check1'] = ass_l.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)
ass_l['Check2'] = ass_l.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)

<ipython-input-87-33bffeb5ccf1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_l['Check1'] = ass_l.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)
<ipython-input-87-33bffeb5ccf1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_l['Check2'] = ass_l.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)


In [88]:
ass_l.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
194,https://nayoo.co/khonkaen/posts/13702,90000000,4500000.0,75000000.0,20.000000,1.200000
112,https://nayoo.co/khonkaen/posts/7955,2600000,32000000.0,2000000.0,12.307692,1.300000
58,https://nayoo.co/khonkaen/posts/9203,160000000,14000000.0,11000000.0,11.428571,14.545455
47,https://nayoo.co/khonkaen/posts/7349,245000000,24000000.0,18000000.0,10.208333,13.611111
7,https://nayoo.co/khonkaen/posts/1726,90000000,18000000.0,70000000.0,5.000000,1.285714
...,...,...,...,...,...,...
182,https://nayoo.co/khonkaen/posts/9598,3900000,4000000.0,4000000.0,1.025641,1.025641
78,https://nayoo.co/khonkaen/posts/3447,980000,1000000.0,800000.0,1.020408,1.225000
100,https://nayoo.co/khonkaen/posts/1663,1485000,1500000.0,1200000.0,1.010101,1.237500
122,https://nayoo.co/khonkaen/posts/12762,650000,650000.0,700000.0,1.000000,1.076923


In [89]:
ass_l.sort_values(by=['Check2'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
58,https://nayoo.co/khonkaen/posts/9203,160000000,14000000.0,11000000.0,11.428571,14.545455
47,https://nayoo.co/khonkaen/posts/7349,245000000,24000000.0,18000000.0,10.208333,13.611111
148,https://nayoo.co/khonkaen/posts/3561,7374550,6000000.0,80000000.0,1.229092,10.848120
188,https://nayoo.co/khonkaen/posts/12250,2300000,2500000.0,24000000.0,1.086957,10.434783
144,https://nayoo.co/khonkaen/posts/3341,1700000,1600000.0,9000000.0,1.062500,5.294118
...,...,...,...,...,...,...
104,https://nayoo.co/khonkaen/posts/1921,8080000,2800000.0,8000000.0,2.885714,1.010000
60,https://nayoo.co/khonkaen/posts/12607,7990000,10000000.0,8000000.0,1.251564,1.001252
158,https://nayoo.co/khonkaen/posts/6732,3500000,2800000.0,3500000.0,1.250000,1.000000
202,https://nayoo.co/khonkaen/posts/16072,750000,500000.0,750000.0,1.500000,1.000000


### MAPE ใหม่

In [90]:
temp = pd.read_excel('คำตอบนักประเมินแก้ไข.xlsx',sheet_name=1)
idl1 = temp[temp['ID ผู้ประเมิน']==1]
idl1 = idl1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idl2 = temp[temp['ID ผู้ประเมิน']==2]
idl2 = idl2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idl1MAPE = mape(idl1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idl1['ราคาทรัพย์']))
idl2MAPE = mape(idl2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idl2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idl1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {idl2MAPE*100:.2f}%')
idl2['ประเมิน2'] = idl2['post id'].map(idl1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_l = idl2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twol = mape(id2_l['ประเมินราคาซื้อ-ขาย'],id2_l['ประเมิน2'])
two2onel = mape(id2_l['ประเมิน2'],id2_l['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twol)+(two2onel))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_l)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 53.94%
นักประเมิน คนที่ 2 = 23.82%
ความผิดพลาดระหว่างนักประเมินทั้ง 42.37%
จำนวนทรัพย์ทั้งหมด 40 ทรัพย์


In [105]:
temp = pd.read_excel('คำตอบนักประเมินaj.xlsx',sheet_name=1)
idl1 = temp[temp['ID ผู้ประเมิน']==1]
idl1 = idl1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idl2 = temp[temp['ID ผู้ประเมิน']==2]
idl2 = idl2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idl1MAPE = mape(idl1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idl1['ราคาทรัพย์']))
idl2MAPE = mape(idl2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idl2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idl1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {idl2MAPE*100:.2f}%')
idl2['ประเมิน2'] = idl2['post id'].map(idl1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_l = idl2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twol = mape(id2_l['ประเมินราคาซื้อ-ขาย'],id2_l['ประเมิน2'])
two2onel = mape(id2_l['ประเมิน2'],id2_l['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twol)+(two2onel))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_l)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 83.48%
นักประเมิน คนที่ 2 = 48.38%
ความผิดพลาดระหว่างนักประเมินทั้ง 58.41%
จำนวนทรัพย์ทั้งหมด 100 ทรัพย์


## Condo

In [91]:
ass_c['Check1']=ass_c.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']),axis=1)

<ipython-input-91-2352dd2a2226>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_c['Check1']=ass_c.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']),axis=1)


In [92]:
ass_c.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,Check1
7,1641.0,4290000.0,2700000.0,1.588889
3,1395.0,1600000.0,2300000.0,1.437500
9,1840.0,3790000.0,2800000.0,1.353571
6,1640.0,1280000.0,1600000.0,1.250000
2,603.0,2438000.0,2800000.0,1.148482
1,36.0,1930000.0,2200000.0,1.139896
8,1650.0,1700000.0,1550000.0,1.096774
4,1443.0,1860000.0,1800000.0,1.033333
5,1609.0,1490000.0,1500000.0,1.006711
0,32.0,3500000.0,3500000.0,1.000000


### MAPE ใหม่

In [93]:
temp = pd.read_excel('คำตอบนักประเมินแก้ไข.xlsx',sheet_name=2)
idc1 = temp[temp['ID ผู้ประเมิน']==1]
idc1 = idt1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])

idc1MAPE = mape(idc1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idc1['ราคาทรัพย์']))

print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idc1MAPE*100:.2f}%')


print(f'จำนวนทรัพย์ทั้งหมด {len(idc1)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 16.21%
จำนวนทรัพย์ทั้งหมด 120 ทรัพย์


## Town home

In [94]:
ass_th['Check1']=ass_th.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']),axis=1)
ass_th['Check2']=ass_th.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']),axis=1)

<ipython-input-94-c9d8ab3947d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_th['Check1']=ass_th.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']),axis=1)
<ipython-input-94-c9d8ab3947d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_th['Check2']=ass_th.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']),axis=1)


In [95]:
ass_th.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
176,https://nayoo.co/khonkaen/posts/3528,2190000.0,350000.0,400000.0,6.257143,5.475000
180,https://nayoo.co/khonkaen/posts/6095,2420000.0,1500000.0,1500000.0,1.613333,1.613333
132,https://nayoo.co/khonkaen/posts/619,1430000.0,950000.0,1000000.0,1.505263,1.430000
1,https://nayoo.co/khonkaen/posts/548,1045000.0,700000.0,1500000.0,1.492857,1.435407
174,https://nayoo.co/khonkaen/posts/3478,2190000.0,1500000.0,2200000.0,1.460000,1.004566
...,...,...,...,...,...,...
228,https://nayoo.co/khonkaen/posts/17923,1990000.0,2000000.0,2200000.0,1.005025,1.105528
167,https://nayoo.co/khonkaen/posts/2829,2490000.0,2500000.0,2300000.0,1.004016,1.082609
220,https://nayoo.co/khonkaen/posts/17545,2800000.0,2800000.0,2800000.0,1.000000,1.000000
200,https://nayoo.co/khonkaen/posts/13411,3200000.0,3200000.0,3200000.0,1.000000,1.000000


In [96]:
ass_th.sort_values(by=['Check2'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
176,https://nayoo.co/khonkaen/posts/3528,2190000.0,350000.0,400000.0,6.257143,5.475000
234,https://nayoo.co/khonkaen/posts/18090,3000000.0,2800000.0,1500000.0,1.071429,2.000000
180,https://nayoo.co/khonkaen/posts/6095,2420000.0,1500000.0,1500000.0,1.613333,1.613333
3,https://nayoo.co/khonkaen/posts/549,935000.0,700000.0,1500000.0,1.335714,1.604278
225,https://nayoo.co/khonkaen/posts/17742,2500000.0,2200000.0,1700000.0,1.136364,1.470588
...,...,...,...,...,...,...
188,https://nayoo.co/khonkaen/posts/8101,3200000.0,3300000.0,3200000.0,1.031250,1.000000
220,https://nayoo.co/khonkaen/posts/17545,2800000.0,2800000.0,2800000.0,1.000000,1.000000
199,https://nayoo.co/khonkaen/posts/12957,1500000.0,1600000.0,1500000.0,1.066667,1.000000
151,https://nayoo.co/khonkaen/posts/1978,2500000.0,2700000.0,2500000.0,1.080000,1.000000


### MAPE ใหม่

In [97]:
temp = pd.read_excel('คำตอบนักประเมินแก้ไข.xlsx',sheet_name=3)
idt1 = temp[temp['ID ผู้ประเมิน']==1]
idt1 = idt1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idt2 = temp[temp['ID ผู้ประเมิน']==2]
idt2 = idt2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
idt1MAPE = mape(idt1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idt1['ราคาทรัพย์']))
idt2MAPE = mape(idt2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(idt2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {idt1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {idt2MAPE*100:.2f}%')
idt2['ประเมิน2'] = idt2['post id'].map(idt1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_t = idt2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twot = mape(id2_t['ประเมินราคาซื้อ-ขาย'],id2_t['ประเมิน2'])
two2onet = mape(id2_t['ประเมิน2'],id2_t['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twot)+(two2onet))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_t)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 12.11%
นักประเมิน คนที่ 2 = 9.24%
ความผิดพลาดระหว่างนักประเมินทั้ง 11.66%
จำนวนทรัพย์ทั้งหมด 61 ทรัพย์


## Shop house

In [98]:
ass_sh['Check1']=ass_sh.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']),axis=1)
ass_sh['Check2']=ass_sh.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']),axis=1)

<ipython-input-98-2bb3ee927996>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_sh['Check1']=ass_sh.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 1']),axis=1)
<ipython-input-98-2bb3ee927996>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_sh['Check2']=ass_sh.apply(lambda x:check_v(x['ราคาทรัพย์'],x['ประเมินโดย 2']),axis=1)


In [99]:
ass_sh.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
9,https://nayoo.co/khonkaen/posts/24,3900000.0,2200000.0,3000000.0,1.772727,1.300000
5,https://nayoo.co/khonkaen/posts/21,22500000.0,13500000.0,20000000.0,1.666667,1.125000
11,https://nayoo.co/khonkaen/posts/30,4800000.0,3600000.0,4000000.0,1.333333,1.200000
15,https://nayoo.co/khonkaen/posts/497,5000000.0,3800000.0,5000000.0,1.315789,1.000000
1,https://nayoo.co/khonkaen/posts/6,7000000.0,6000000.0,8000000.0,1.166667,1.142857
7,https://nayoo.co/khonkaen/posts/22,3500000.0,4000000.0,4000000.0,1.142857,1.142857
19,https://nayoo.co/khonkaen/posts/499,9000000.0,8000000.0,10000000.0,1.125000,1.111111
13,https://nayoo.co/khonkaen/posts/35,2400000.0,2500000.0,2100000.0,1.041667,1.142857
3,https://nayoo.co/khonkaen/posts/17,12000000.0,12000000.0,12000000.0,1.000000,1.000000
17,https://nayoo.co/khonkaen/posts/498,7000000.0,7000000.0,8000000.0,1.000000,1.142857


In [100]:
ass_sh.sort_values(by=['Check2'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
9,https://nayoo.co/khonkaen/posts/24,3900000.0,2200000.0,3000000.0,1.772727,1.300000
11,https://nayoo.co/khonkaen/posts/30,4800000.0,3600000.0,4000000.0,1.333333,1.200000
1,https://nayoo.co/khonkaen/posts/6,7000000.0,6000000.0,8000000.0,1.166667,1.142857
7,https://nayoo.co/khonkaen/posts/22,3500000.0,4000000.0,4000000.0,1.142857,1.142857
13,https://nayoo.co/khonkaen/posts/35,2400000.0,2500000.0,2100000.0,1.041667,1.142857
17,https://nayoo.co/khonkaen/posts/498,7000000.0,7000000.0,8000000.0,1.000000,1.142857
5,https://nayoo.co/khonkaen/posts/21,22500000.0,13500000.0,20000000.0,1.666667,1.125000
19,https://nayoo.co/khonkaen/posts/499,9000000.0,8000000.0,10000000.0,1.125000,1.111111
3,https://nayoo.co/khonkaen/posts/17,12000000.0,12000000.0,12000000.0,1.000000,1.000000
15,https://nayoo.co/khonkaen/posts/497,5000000.0,3800000.0,5000000.0,1.315789,1.000000


### MAPE ใหม่

In [101]:
temp = pd.read_excel('คำตอบนักประเมินแก้ไข.xlsx',sheet_name=4)
ids1 = temp[temp['ID ผู้ประเมิน']==1]
ids1 = ids1.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
ids2 = temp[temp['ID ผู้ประเมิน']==2]
ids2 = ids2.dropna(subset=['ราคาทรัพย์','ประเมินราคาซื้อ-ขาย'])
ids1MAPE = mape(ids1['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(ids1['ราคาทรัพย์']))
ids2MAPE = mape(ids2['ประเมินราคาซื้อ-ขาย'],pd.to_numeric(ids2['ราคาทรัพย์']))
print(f'ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ\nบ้าน\nนักประเมิน คนที่ 1 = {ids1MAPE*100:.2f}%\nนักประเมิน คนที่ 2 = {ids2MAPE*100:.2f}%')
ids2['ประเมิน2'] = ids2['post id'].map(ids1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
id2_s = idt2.dropna(subset=['ประเมินราคาซื้อ-ขาย','ประเมิน2','ราคาทรัพย์'])
one2twos = mape(id2_s['ประเมินราคาซื้อ-ขาย'],id2_s['ประเมิน2'])
two2ones = mape(id2_s['ประเมิน2'],id2_s['ประเมินราคาซื้อ-ขาย'])
print(f'ความผิดพลาดระหว่างนักประเมินทั้ง {((one2twos)+(two2ones))*50:.2f}%')
print(f'จำนวนทรัพย์ทั้งหมด {len(id2_s)} ทรัพย์')

ความแตกต่างระหว่างราคาประกาศขายและราคาประเมินของ
บ้าน
นักประเมิน คนที่ 1 = 25.45%
นักประเมิน คนที่ 2 = 12.43%
ความผิดพลาดระหว่างนักประเมินทั้ง 11.66%
จำนวนทรัพย์ทั้งหมด 61 ทรัพย์


# HW3